In [1]:
cd '~/cloudFinder/'

/home/antoine/cloudFinder


In [2]:
import jupyterlab_dash
import dash
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output
import feather as ft 
import pandas as pd
import base64
import dash_table
from io import BytesIO
from skimage import measure


import plotly.graph_objects as go
import plotly.offline as po

from src.image.utils import load_image
from src.image.transform import resize_image
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
viewer = jupyterlab_dash.AppViewer(port=9998)

======================================================================================================

In [3]:
MODEL_PATH = 'data/process/sky_segmentation/'

======================================================================================================

In [4]:
df_score = ft.read_dataframe(MODEL_PATH + '/score_per_image.ft')
df_score.head()

,img_name,prediction_label,metric,score
0,labelme_aacpgupgzvdjapw.jpg,is_sky_PREDICTED,accuracy,0.874812
1,labelme_aacpgupgzvdjapw.jpg,is_sky_PREDICTED,precision,0.780409
2,labelme_aacpgupgzvdjapw.jpg,is_sky_PREDICTED,recall,0.585103
3,labelme_aacpgupgzvdjapw.jpg,is_sky_PREDICTED_COR,accuracy,0.882331
4,labelme_aacpgupgzvdjapw.jpg,is_sky_PREDICTED_COR,precision,0.902214


In [5]:
df_cv = ft.read_dataframe(MODEL_PATH + '/cv.ft')
df_cv['is_sky'] = df_cv['is_sky'].astype(int)
df_cv.head()

,img_name,is_sky,is_sky_PREDICTED,fold,is_sky_PREDICTED_COR
0,labelme_aacpgupgzvdjapw.jpg,0,1,1,0
1,labelme_aacpgupgzvdjapw.jpg,0,0,1,0
2,labelme_aacpgupgzvdjapw.jpg,0,1,1,0
3,labelme_aacpgupgzvdjapw.jpg,0,1,1,0
4,labelme_aacpgupgzvdjapw.jpg,0,1,1,0


In [6]:
df_meta = ft.read_dataframe(MODEL_PATH + '/traindataset_meta.ft')
df_meta.head()

,size_h,size_l,sky_coverage,is_used,img_name
0,100,133,0.216015,True,labelme_aacpgupgzvdjapw.jpg
1,100,133,0.103534,True,labelme_aajuldidvlcyzhv.jpg
2,0,0,0.002105,False,labelme_aajvoauwthyaehf.jpg
3,100,133,0.337744,True,labelme_aalbzqrimafwbiv.jpg
4,0,0,0.022632,False,labelme_aatpnkdmtvmagyh.jpg


In [7]:
tmp = df_score.groupby(['metric', 'img_name', 'prediction_label']).mean().loc["accuracy", 'score'].unstack(level=1).reset_index()
tmp = pd.merge(tmp, df_meta, right_on='img_name', left_on='img_name', how='outer')
tmp = tmp[['img_name', 'is_sky_PREDICTED', 'is_sky_PREDICTED_COR', 'is_used', 'sky_coverage']]
tmp
#tmp.columns = ["Image", "Score", "Corrected Score", "Is Used", "Sky Coverage"]

,img_name,is_sky_PREDICTED,is_sky_PREDICTED_COR,is_used,sky_coverage
0,labelme_aacpgupgzvdjapw.jpg,0.874812,0.882331,True,0.216015
1,labelme_aajuldidvlcyzhv.jpg,0.952932,0.972782,True,0.103534
2,labelme_aalbzqrimafwbiv.jpg,0.986391,0.984361,True,0.337744
3,labelme_aaybneesrunupnk.jpg,0.945489,0.973534,True,0.241729
4,labelme_abxjjdteigmqnyg.jpg,0.865865,0.859624,True,0.631353
...,...,...,...,...,...
7550,sun_dsuaivkzkxmfrxwt.jpg,NaN,NaN,False,0.002556
7551,sun_dubsjhuabbmdclhc.jpg,NaN,NaN,False,0.001579
7552,sun_dwiizofpizmsybam.jpg,NaN,NaN,False,0.042030
7553,sun_dwksaxsgjjqeaetf.jpg,NaN,NaN,False,0.031203


In [8]:
all_img = set(df_cv['img_name'])
all_metric = set(['accuracy', 'recall', 'precision'])

In [9]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        html.Div([html.H1(children="Sky segmentation"),], className="row"),
        html.Div(
            [
                html.Div(
                    [
                        html.H6(children="Metric"),
                        dcc.Dropdown(
                            options=[{"label": i, "value": i} for i in all_metric],
                            value="accuracy",
                            multi=False,
                            id="metric",
                        ),
                        html.H6(children="Image score"),
                        dash_table.DataTable(
                            id="score_table",
                            columns=[
                                {"name": i, "id": i}
                                for i in ["Image", "Score", "Corrected Score", "Is Used", "Sky Coverage"]
                            ],
                            sort_action="native",
                            page_size=10,
                            style_table={"maxHeight": "50", "overflowY": "scroll"},
                        ),
                    ],
                    className="four columns",
                ),
                html.Div(
                    [html.H6(children="Score"), dcc.Graph(id="box_plot"),],
                    className="eight columns",
                ),
            ],
            className="row",
        ),
        html.Div(
            [
                html.Div(
                    [
                        html.H6(children="image name"),
                        dcc.Dropdown(
                            options=[{"label": i, "value": i} for i in all_img],
                            value="labelme_aacpgupgzvdjapw.jpg",
                            multi=False,
                            id="img_name",
                        ),
                        html.H6(children="With correction"),
                        dcc.RadioItems(
                            id="corr",
                            options=[
                                {"label": i, "value": i} for i in ["True", "False"]
                            ],
                            value="False",
                            labelStyle={"display": "inline-block"},
                        ),
                        html.H6(children="confusion matrix"),
                        dcc.Markdown(
                            children="""
                                            |          | pos      | neg      |
                                            |----------|----------|----------|
                                            | Pred Pos | 0   | 0  | 
                                            | Pred Neg | 0   | 0  | 
                                        """,
                            id="confusion_matrix",
                        ),
                    ],
                    className="four columns",
                ),
                html.Div(
                    [html.H6(children="Image"), html.Img(src="", id="img_plot"),],
                    className="eight columns",
                ),
            ],
            className="row",
        ),
    ],
    className="ten columns offset-by-one",
)

In [10]:
@app.callback(
    Output("box_plot", "figure"), [Input("metric", "value")]
)
def update_plot(metric):
    tmp = df_score.groupby(['metric', 'img_name', 'prediction_label']).mean().loc[metric, 'score'].unstack(level=1).reset_index()
    data = []
    data.append(go.Box(y=tmp["is_sky_PREDICTED"], name= "without_cor"))
    data.append(go.Box(y=tmp["is_sky_PREDICTED_COR"], name="with_cor"))
    return {"data": data}

In [11]:
@app.callback(
    Output("confusion_matrix", "children"),
    [Input("img_name", "value"), Input("corr", "value")],
)
def update_confusion_matrix(img_name, with_correction):

    if with_correction == "True":
        pred_label = "is_sky_PREDICTED_COR"
    else:
        pred_label = "is_sky_PREDICTED"

    tmp = df_cv[df_cv["img_name"] == img_name].copy()

    tn, fp, fn, tp = confusion_matrix(tmp["is_sky"], tmp[pred_label]).ravel()

    txt = """
        |          | pos      | neg      |
        |----------|----------|----------|
        | Pred Pos | {}   | {}  | 
        | Pred Neg | {}   | {}    |
        """.format(
        tp, fp, fn, tn
    )
    return txt

In [12]:
@app.callback(Output('score_table', 'data'), [Input('metric', 'value')])
def update_table(metric):
    tmp = df_score.groupby(['metric', 'img_name', 'prediction_label']).mean().loc[metric, 'score'].unstack(level=1).reset_index()
    tmp = pd.merge(tmp, df_meta, right_on='img_name', left_on='img_name', how='outer')
    tmp = tmp[['img_name', 'is_sky_PREDICTED', 'is_sky_PREDICTED_COR', 'is_used', 'sky_coverage']]
    tmp.columns = ["Image", "Score", "Corrected Score", "Is Used", "Sky Coverage"]
    return tmp.to_dict("rows")


In [13]:
@app.callback(
    Output("img_plot", "src"), [Input("img_name", "value"), Input("corr", "value")]
)
def update_image(img_name, with_correction):

    if with_correction == "True":
        pred_label = "is_sky_PREDICTED_COR"
    else:
        pred_label = "is_sky_PREDICTED"

    size_h, size_l = df_meta[df_meta.img_name == img_name][["size_h", "size_l"]].iloc[0]

    df_cv_image = df_cv[df_cv.img_name == img_name].copy()

    sky = df_cv_image["is_sky"].values.reshape((size_h, size_l))
    sky_contours = measure.find_contours(sky, 0)
    
    sky_predicted = df_cv_image[pred_label].values.reshape((size_h, size_l))
    sky_predicted_contours = measure.find_contours(sky_predicted, 0)

    img = load_image("data/img/sky_segmentation/", img_name)
    img_resized = resize_image(img, 100)

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img_resized)
    for n, c in enumerate(sky_contours):
        ax.plot(c[:, 1], c[:, 0], linewidth=3, color="r")
    for n, c in enumerate(sky_predicted_contours):
        ax.plot(c[:, 1], c[:, 0], linewidth=3, color="y")
    figfile = BytesIO()
    plt.savefig(figfile, format="png")
    figdata_png = base64.b64encode(figfile.getvalue()).decode()

    src = "data:image/png;base64,{}".format(figdata_png)

    return src

In [14]:
viewer.stderr_queue.write('Running on'.encode('utf-8'))
viewer.show(app)

In [ ]:
viewer.terminate()